In [3]:
!mkdir zoo; cd zoo; touch __init__.py

In [1]:
import warnings ; warnings.filterwarnings('ignore')
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from IPython.display import display
from collections import namedtuple, deque
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from itertools import cycle, count
from textwrap import wrap

import matplotlib
import subprocess
import os.path
import tempfile
import random
import base64
import pprint
import glob
import time
import json
import sys
import gym
import io
import os
import gc

from gym import wrappers
from subprocess import check_output
from IPython.display import HTML

from zoo.policy_gradient_agents import *
from zoo.actor_critic_agents import *
from zoo.utils import *

SEEDS = (12, 34, 56)

%matplotlib inline

In [ ]:
reinforce_results = []
best_agent, best_eval_score = None, float('-inf')
for seed in SEEDS:
    environment_settings = {
        'env_name': 'CartPole-v1',
        'gamma': 1.00,
        'max_minutes': 5,
        'max_episodes': 5000,
        'goal_mean_100_reward': 475
    }

    policy_model_fn = lambda nS, nA: FCDAP(nS, nA, hidden_dims=(128,64))
    policy_optimizer_fn = lambda net, lr: optim.Adam(net.parameters(), lr=lr)
    policy_optimizer_lr = 0.0005

    env_name, gamma, max_minutes, \
    max_episodes, goal_mean_100_reward = environment_settings.values()
    agent = REINFORCE(policy_model_fn, policy_optimizer_fn, policy_optimizer_lr)

    make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
    # make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name, unwrapped=True)
    # make_env_fn, make_env_kargs = get_make_env_fn(
    #     env_name=env_name, addon_wrappers=[MCCartPole,])
    result, final_eval_score, training_time, wallclock_time = agent.train(
        make_env_fn, make_env_kargs, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward)
    reinforce_results.append(result)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = agent
reinforce_results = np.array(reinforce_results)
_ = BEEP()

el 00:00:00, ep 0000, ts 000020, ar 10 020.0±000.0, 100 020.0±000.0, ex 100 0.5±0.0, ev 012.0±000.0
el 00:01:00, ep 0403, ts 030072, ar 10 207.1±092.1, 100 164.4±086.7, ex 100 0.3±0.0, ev 296.0±132.1
el 00:02:00, ep 0598, ts 072569, ar 10 334.5±106.2, 100 231.2±106.6, ex 100 0.3±0.0, ev 311.4±148.6
el 00:02:47, ep 0703, ts 107979, ar 10 373.6±129.8, 100 334.2±127.6, ex 100 0.3±0.0, ev 475.5±056.7
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 500.00±0.00 in 121.52s training time, 177.81s wall-clock time.

el 00:00:00, ep 0000, ts 000019, ar 10 019.0±000.0, 100 019.0±000.0, ex 100 0.5±0.0, ev 010.0±000.0
el 00:01:00, ep 0499, ts 025611, ar 10 106.4±040.0, 100 117.3±074.0, ex 100 0.3±0.0, ev 227.4±115.6
el 00:02:00, ep 0690, ts 064263, ar 10 225.8±057.1, 100 195.4±092.2, ex 100 0.3±0.0, ev 306.6±137.2
el 00:03:00, ep 0843, ts 106945, ar 10 352.7±039.5, 100 300.9±124.4, ex 100 0.3±0.0, ev 406.6±134.0
el 00:04:01, ep 1029, ts 148419, ar 10 356.9±147.2, 100 261.8±1

In [ ]:
vpg_results = []
best_agent, best_eval_score = None, float('-inf')
for seed in SEEDS:
    environment_settings = {
        'env_name': 'CartPole-v1',
        'gamma': 1.00,
        'max_minutes': 10,
        'max_episodes': 10000,
        'goal_mean_100_reward': 475
    }

    policy_model_fn = lambda nS, nA: FCDAP(nS, nA, hidden_dims=(128,64))
    policy_model_max_grad_norm = 1
    policy_optimizer_fn = lambda net, lr: optim.Adam(net.parameters(), lr=lr)
    policy_optimizer_lr = 0.0005

    value_model_fn = lambda nS: FCV(nS, hidden_dims=(256,128))
    value_model_max_grad_norm = float('inf')
    value_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)
    value_optimizer_lr = 0.0007

    entropy_loss_weight = 0.001

    env_name, gamma, max_minutes, \
    max_episodes, goal_mean_100_reward = environment_settings.values()
    agent = VPG(policy_model_fn, 
                policy_model_max_grad_norm, 
                policy_optimizer_fn, 
                policy_optimizer_lr,
                value_model_fn, 
                value_model_max_grad_norm, 
                value_optimizer_fn, 
                value_optimizer_lr, 
                entropy_loss_weight)

    make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
    result, final_eval_score, training_time, wallclock_time = agent.train(
        make_env_fn, make_env_kargs, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward)
    vpg_results.append(result)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = agent
vpg_results = np.array(vpg_results)
_ = BEEP()

el 00:00:00, ep 0000, ts 000015, ar 10 015.0±000.0, 100 015.0±000.0, ex 100 0.7±0.0, ev 022.0±000.0
el 00:01:00, ep 0248, ts 019771, ar 10 159.1±062.9, 100 134.8±086.1, ex 100 0.3±0.0, ev 345.5±126.5
el 00:02:00, ep 0359, ts 045432, ar 10 307.4±077.9, 100 238.0±114.0, ex 100 0.3±0.0, ev 405.8±114.0
el 00:03:01, ep 0451, ts 077933, ar 10 436.1±109.9, 100 349.5±111.3, ex 100 0.3±0.0, ev 468.3±055.0
el 00:03:32, ep 0489, ts 096264, ar 10 493.7±011.1, 100 415.0±104.3, ex 100 0.3±0.0, ev 476.1±046.5
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 499.63±3.68 in 169.36s training time, 221.81s wall-clock time.

el 00:00:00, ep 0000, ts 000016, ar 10 016.0±000.0, 100 016.0±000.0, ex 100 0.5±0.0, ev 010.0±000.0
el 00:01:00, ep 0286, ts 026565, ar 10 257.7±068.4, 100 178.5±105.3, ex 100 0.3±0.0, ev 370.0±121.3
el 00:02:00, ep 0393, ts 056297, ar 10 435.7±073.3, 100 282.5±110.7, ex 100 0.3±0.0, ev 374.9±123.9
el 00:03:01, ep 0477, ts 088902, ar 10 483.1±050.7, 100 393.7±1

In [2]:
a3c_results = []
best_agent, best_eval_score = None, float('-inf')
def policy_model_fn (nS, nA):
    return FCDAP(nS, nA, hidden_dims=(128,64))
policy_model_max_grad_norm = 1
def policy_optimizer_fn(net, lr):
    return SharedAdam(net.parameters(), lr=lr)
policy_optimizer_lr = 0.0005

def value_model_fn(nS):
    return FCV(nS, hidden_dims=(256,128))
value_model_max_grad_norm = float('inf')
def value_optimizer_fn(net, lr):
    return SharedRMSprop(net.parameters(), lr=lr)
value_optimizer_lr = 0.0007

def make_env_fn(env_name, seed=None, render=None, record=False,
                unwrapped=False, monitor_mode=None,
                inner_wrappers=None, outer_wrappers=None):
    mdir = tempfile.mkdtemp()
    env = None
    if render:
        try:
            env = gym.make(env_name, render=render)
        except:
            pass
    if env is None:
        env = gym.make(env_name)
    if seed is not None: env.seed(seed)
    env = env.unwrapped if unwrapped else env
    if inner_wrappers:
        for wrapper in inner_wrappers:
            env = wrapper(env)
    env = wrappers.Monitor(
        env, mdir, force=True,
        mode=monitor_mode,
        video_callable=lambda e_idx: record) if monitor_mode else env
    if outer_wrappers:
        for wrapper in outer_wrappers:
            env = wrapper(env)
    return env

for seed in SEEDS:
    environment_settings = {
        'env_name': 'CartPole-v1',
        'gamma': 1.00,
        'max_minutes': 5,
        'max_episodes': 2500,
        'goal_mean_100_reward': 475
    }
    

    entropy_loss_weight = 0.001

    max_n_steps = 50
    n_workers = 1

    env_name, gamma, max_minutes, \
    max_episodes, goal_mean_100_reward = environment_settings.values()
    agent = A3C(policy_model_fn,
                policy_model_max_grad_norm, 
                policy_optimizer_fn, 
                policy_optimizer_lr,
                value_model_fn,
                value_model_max_grad_norm,
                value_optimizer_fn, 
                value_optimizer_lr,
                entropy_loss_weight, 
                max_n_steps,
                n_workers)

    #make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
    make_env_kargs = {'env_name':env_name}
    result, final_eval_score, training_time, wallclock_time = agent.train(
        make_env_fn, make_env_kargs, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward)
    a3c_results.append(result)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = agent
a3c_results = np.array(a3c_results)
_ = BEEP()

el 00:00:00, ep 0000, ts 000015, ar 10 015.0±000.0, 100 015.0±000.0, ex 100 0.7±0.0, ev 022.0±000.0
el 00:01:00, ep 0310, ts 039161, ar 10 322.8±098.8, 100 261.4±133.1, ex 100 0.3±0.0, ev 308.5±144.7
el 00:02:00, ep 0455, ts 089022, ar 10 220.3±010.9, 100 379.2±129.2, ex 100 0.2±0.0, ev 382.6±124.8
el 00:02:57, ep 0564, ts 138018, ar 10 500.0±000.0, 100 471.5±074.5, ex 100 0.2±0.0, ev 475.2±068.6
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 500.00±0.00 in 153.98s training time, 177.61s wall-clock time.

el 00:00:00, ep 0000, ts 000016, ar 10 016.0±000.0, 100 016.0±000.0, ex 100 0.5±0.0, ev 010.0±000.0
el 00:01:00, ep 0328, ts 036103, ar 10 199.9±033.5, 100 168.3±105.7, ex 100 0.3±0.0, ev 220.7±143.6
el 00:02:00, ep 0500, ts 083437, ar 10 500.0±000.0, 100 365.7±138.4, ex 100 0.2±0.0, ev 422.2±116.7
el 00:03:00, ep 0674, ts 131401, ar 10 371.1±107.4, 100 302.7±179.2, ex 100 0.2±0.0, ev 309.0±175.6
el 00:03:26, ep 0719, ts 153901, ar 10 500.0±000.0, 100 472.6±0

In [2]:
gae_results = []
best_agent, best_eval_score = None, float('-inf')
for seed in SEEDS:
    environment_settings = {
        'env_name': 'CartPole-v1',
        'gamma': 0.99,
        'max_minutes': 10,
        'max_episodes': 10000,
        'goal_mean_100_reward': 475
    }

    policy_model_fn = lambda nS, nA: FCDAP(nS, nA, hidden_dims=(128,64))
    policy_model_max_grad_norm = 1
    policy_optimizer_fn = lambda net, lr: SharedAdam(net.parameters(), lr=lr)
    policy_optimizer_lr = 0.0005

    value_model_fn = lambda nS: FCV(nS, hidden_dims=(256,128))
    value_model_max_grad_norm = float('inf')
    value_optimizer_fn = lambda net, lr: SharedRMSprop(net.parameters(), lr=lr)
    value_optimizer_lr = 0.0007

    entropy_loss_weight = 0.001

    max_n_steps = 50
    n_workers = 8
    tau = 0.95

    env_name, gamma, max_minutes, \
    max_episodes, goal_mean_100_reward = environment_settings.values()
    agent = GAE(policy_model_fn,
                policy_model_max_grad_norm, 
                policy_optimizer_fn, 
                policy_optimizer_lr,
                value_model_fn,
                value_model_max_grad_norm,
                value_optimizer_fn, 
                value_optimizer_lr, 
                entropy_loss_weight,
                max_n_steps,
                n_workers,
                tau)

    make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
    result, final_eval_score, training_time, wallclock_time = agent.train(
        make_env_fn, make_env_kargs, seed, gamma, max_minutes, max_episodes, goal_mean_100_reward)
    gae_results.append(result)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = agent
gae_results = np.array(gae_results)
_ = BEEP()

el 00:00:00, ep 0000, ts 000015, ar 10 015.0±000.0, 100 015.0±000.0, ex 100 0.7±0.0, ev 011.0±000.0
el 00:01:01, ep 0409, ts 043822, ar 10 139.5±125.5, 100 220.6±104.9, ex 100 0.3±0.1, ev 370.4±124.2
el 00:01:59, ep 0548, ts 101746, ar 10 500.0±000.0, 100 446.2±091.4, ex 100 0.2±0.0, ev 485.4±051.3
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 499.65±3.48 in 799.61s training time, 119.04s wall-clock time.

el 00:00:00, ep 0000, ts 000016, ar 10 016.0±000.0, 100 016.0±000.0, ex 100 0.5±0.0, ev 011.0±000.0
el 00:01:02, ep 0478, ts 041655, ar 10 224.7±077.4, 100 197.1±084.9, ex 100 0.3±0.0, ev 307.5±149.3
el 00:01:44, ep 0587, ts 079503, ar 10 388.6±125.4, 100 356.8±120.4, ex 100 0.3±0.0, ev 475.9±063.6
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 441.28±61.34 in 706.53s training time, 105.61s wall-clock time.

el 00:00:00, ep 0000, ts 000012, ar 10 012.0±000.0, 100 012.0±000.0, ex 100 0.2±0.0, ev 009.0±000.0
el 00:01:02, ep 0490, ts 

In [2]:
a2c_results = []
best_agent, best_eval_score = None, float('-inf')
for seed in SEEDS:
    environment_settings = {
        'env_name': 'CartPole-v1',
        'gamma': 0.99,
        'max_minutes': 10,
        'max_episodes': 10000,
        'goal_mean_100_reward': 475
    }
    
    ac_model_fn = lambda nS, nA: FCAC(nS, nA, hidden_dims=(256,128))
    ac_model_max_grad_norm = 1
    # ac_optimizer_fn = lambda net, lr: optim.Adam(net.parameters(), lr=lr)
    ac_optimizer_fn = lambda net, lr: optim.RMSprop(net.parameters(), lr=lr)
    ac_optimizer_lr = 0.001

    policy_loss_weight = 1.0
    value_loss_weight = 0.6

    entropy_loss_weight = 0.001

    max_n_steps = 10
    n_workers = 8
    tau = 0.95

    env_name, gamma, max_minutes, \
    max_episodes, goal_mean_100_reward = environment_settings.values()
    agent = A2C(ac_model_fn, 
                ac_model_max_grad_norm,
                ac_optimizer_fn,
                ac_optimizer_lr,
                policy_loss_weight,
                value_loss_weight,
                entropy_loss_weight,
                max_n_steps,
                n_workers,
                tau)

    make_envs_fn = lambda mef, mea, s, n: MultiprocessEnv(mef, mea, s, n) 
    make_env_fn, make_env_kargs = get_make_env_fn(env_name=env_name)
    result, final_eval_score, training_time, wallclock_time = agent.train(make_envs_fn,
                                                                          make_env_fn,
                                                                          make_env_kargs,
                                                                          seed,
                                                                          gamma,
                                                                          max_minutes,
                                                                          max_episodes,
                                                                          goal_mean_100_reward)
    a2c_results.append(result)
    if final_eval_score > best_eval_score:
        best_eval_score = final_eval_score
        best_agent = agent
a2c_results = np.array(a2c_results)
_ = BEEP()

el 00:00:00, ep 0000, ts 000014, ar 10 014.0±000.0, 100 014.0±000.0, ex 100 0.5±0.0, ev 010.0±000.0
el 00:01:00, ep 0702, ts 103659, ar 10 440.9±090.8, 100 324.6±161.4, ex 100 0.2±0.0, ev 443.6±088.7
el 00:01:18, ep 0775, ts 139752, ar 10 500.0±000.0, 100 460.7±092.7, ex 100 0.2±0.0, ev 476.5±057.3
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 499.70±2.26 in 589.82s training time, 86.82s wall-clock time.

el 00:00:00, ep 0000, ts 000014, ar 10 014.0±000.0, 100 014.0±000.0, ex 100 0.4±0.0, ev 011.0±000.0
el 00:01:00, ep 0703, ts 107602, ar 10 471.7±084.9, 100 407.4±141.2, ex 100 0.2±0.0, ev 470.0±057.0
el 00:01:03, ep 0714, ts 112568, ar 10 466.1±101.7, 100 423.2±135.8, ex 100 0.2±0.0, ev 476.7±050.8
--> reached_goal_mean_reward ✓
Training complete.
Final evaluation score 500.00±0.00 in 473.37s training time, 71.51s wall-clock time.

el 00:00:00, ep 0000, ts 000009, ar 10 009.0±000.0, 100 009.0±000.0, ex 100 0.9±0.0, ev 010.0±000.0
el 00:00:53, ep 0648, ts 096